In [1]:
USE_GPU = True

In [2]:

SCALED_COLS =[
    'timestamp',
    '30_minute_session_count',
    '5_minute_session_count',
    'task_within_session_count',
]


ENCODED_COLS = [
    'user_id',
    'project_id',
    'country'
]


GROUPBY_COLS = ['user_id']

TIMESTAMP_INDEX = 1

INITIAL_LOAD_COLUMNS = ENCODED_COLS +  ['label', 'date_time'] +  [col for col in SCALED_COLS if 'timestamp' not in col and 'project_count' not in col]

In [52]:

if USE_GPU: 
    import cupy as np
    import numpy
    import pandas
    import cudf as pd
    
    import dask_cudf as dd
    import pandas as cpu_pd
    from cuml.preprocessing import MinMaxScaler
else:
    import numpy as np
    import numpy
    import pandas as pd
    import dask.dataframe as dd
    from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
import matplotlib.pyplot as plt
import glob
# numpy.set_printoptions(precision=4, suppress=True, linewidth=200)
# pandas.options.display.max_columns = 100
pandas.options.display.max_rows = 100
# pandas.options.display.width = 200
# pandas.options.display.chop_threshold = 4


In [4]:
files = glob.iglob('frequency_encoded_data/*.csv')
files = sorted(list(files))

In [5]:
df = dd.read_csv(files, usecols=INITIAL_LOAD_COLUMNS).compute()


In [6]:
df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(by=['date_time'])

In [49]:
test_bug = df[df['user_id'] == 2354175.0]

In [53]:
test_bug.head(500)

,project_id,user_id,country,date_time,5_minute_session_count,30_minute_session_count,label,task_within_session_count
298613,2218.0,2354175.0,Finland,2021-11-21 14:07:35,1.0,1.0,True,1.0
298626,2218.0,2354175.0,Finland,2021-11-21 14:07:45,1.0,1.0,True,2.0
298636,2218.0,2354175.0,Finland,2021-11-21 14:07:52,1.0,1.0,True,3.0
298724,2218.0,2354175.0,Finland,2021-11-21 14:08:57,1.0,1.0,True,4.0
298746,2218.0,2354175.0,Finland,2021-11-21 14:09:10,1.0,1.0,True,5.0
298756,2218.0,2354175.0,Finland,2021-11-21 14:09:20,1.0,1.0,True,6.0
298777,2218.0,2354175.0,Finland,2021-11-21 14:09:31,1.0,1.0,True,7.0
298793,2218.0,2354175.0,Finland,2021-11-21 14:09:44,1.0,1.0,True,8.0
298848,2218.0,2354175.0,Finland,2021-11-21 14:10:25,1.0,1.0,True,9.0
298899,2218.0,2354175.0,Finland,2021-11-21 14:10:57,1.0,1.0,True,10.0


In [7]:
session_time = df.groupby(['user_id', '30_minute_session_count']).agg({'date_time': ['min', 'max']}).reset_index()

In [8]:
session_time['minutes_in_session'] = (session_time['date_time']['max'] - session_time['date_time']['min'])

In [9]:
session_time = session_time.to_pandas()

In [10]:
session_time['minutes_in_session'] = (session_time['minutes_in_session'].dt.total_seconds() / 60).astype(int)

In [48]:
session_time[session_time['minutes_in_session'] > 600].sort_values(by=['minutes_in_session'], ascending=False)

user_id 30_minute_session_count           date_time  \
                                                          min   
591763  2354175.0                     1.0 2021-11-21 14:07:35   
61662   2021653.0                     2.0 2021-12-03 14:34:33   
122337  2077142.0                    11.0 2021-11-22 11:01:31   
294513  2326447.0                     6.0 2022-01-02 08:01:49   
496319  2047747.0                     6.0 2022-01-04 19:06:24   
...           ...                     ...                 ...   
129779  1870670.0                   313.0 2022-07-15 14:37:59   
518382  1792337.0                   308.0 2022-01-24 06:17:13   
47601   1216856.0                    20.0 2022-02-02 19:42:25   
60200   2483000.0                    16.0 2022-06-23 00:57:05   
11231   2316079.0                    36.0 2021-12-11 12:54:09   

                           minutes_in_session  
                       max                     
591763 2022-08-05 13:43:52             370056  
61662  2022-07-15 13:15:27             322480  
122337 2022-06-27 12:06:01             312544  
294513 2022-07-05 11:54:27             265192  
496319 2022-07-05 18:36:20             262049  
...                    ...                ...  
129779 2022-07-16 00:46:38                608  
518382 2022-01-24 16:26:04                608  
47601  2022-02-03 05:48:57                606  
60200  2022-06-23 11:02:54                605  
11231  2021-12-11 22:58:50                604  

[1833 rows x 5 columns]

In [12]:
session_time['minutes_in_session'] = session_time['minutes_in_session'].apply(lambda x: 1 if x == 0 else x)

In [39]:
hist_bins = numpy.concatenate((numpy.array([10, 30]), numpy.arange(60, 630, 30)))

In [41]:
hist = numpy.histogram(session_time['minutes_in_session'], bins=hist_bins)

In [42]:
hist

(array([181662, 120441,  44484,  19059,   8876,   4882,   3168,   2439,
          1424,    967,    525,    328,    191,    112,     85,     64,
            63,     49,     25,     29]),
 array([ 10,  30,  60,  90, 120, 150, 180, 210, 240, 270, 300, 330, 360,
        390, 420, 450, 480, 510, 540, 570, 600]))

In [43]:
counts, bins = hist

In [46]:
display(counts.shape)
display(bins.shape)
greater_600 = session_time.shape[0] - numpy.sum(counts)
display(greater_600)

(20,)

(21,)

261686